# Online Prediction with XGBoost on AI Platform
This notebook uses the [Census Income Data Set](https://archive.ics.uci.edu/ml/datasets/Census+Income) to create a simple XGBoost model, upload the model to Ai Platform, and query it for predictions. 

# How to bring your model to AI Platform
Getting your model ready for predictions can be done in 5 steps:
1. Save your model to a file
2. Upload the saved model to [Google Cloud Storage](https://cloud.google.com/storage)
3. Create a model resource on AI Platform
4. Create a model version (linking your XGBoost model)
5. Make an online prediction

# Prerequisites
Before we begin, let’s cover some of the different tools you’ll use to get online prediction up and running on AI Platform. 

[Google Cloud Platform](https://cloud.google.com/) (GCP) lets you build and host applications and websites, store data, and analyze data on Google's scalable infrastructure.

[AI Platform](https://cloud.google.com/ml-engine/) is a managed service that enables you to easily build machine learning models that work on any type of data, of any size.

[Google Cloud Storage](https://cloud.google.com/storage/) (GCS) is a unified object storage for developers and enterprises, from live data serving to data analytics/ML to data archiving.

[Cloud SDK](https://cloud.google.com/sdk/) is a command line tool which allows you to interact with Google Cloud products. In order to run this notebook, make sure that Cloud SDK is [installed](https://cloud.google.com/sdk/downloads) in the same environment as your Jupyter kernel.


# Part 0: Setup
* [Create a project on GCP](https://cloud.google.com/resource-manager/docs/creating-managing-projects)
* [Create a GCS Bucket](https://cloud.google.com/storage/docs/quickstart-console)
* [Enable AI Platform Training and Prediction and Compute Engine APIs](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component&_ga=2.217405014.1312742076.1516128282-1417583630.1516128282)
* [Install Cloud SDK](https://cloud.google.com/sdk/downloads)
* [Install XGBoost](http://xgboost.readthedocs.io/en/latest/build.html)
* [Install scikit-learn](http://scikit-learn.org/stable/install.html)
* [Install NumPy](https://docs.scipy.org/doc/numpy/user/install.html)
* [Install pandas](https://pandas.pydata.org/pandas-docs/stable/install.html)
* [Install Google API Python Client](https://github.com/google/google-api-python-client)


These variables will be needed for the following steps.

In the cell below, **replace** the following highlighted elements:
* `<YOUR_PROJECT_ID>` - with your project's id. Use the PROJECT_ID that matches your Google Cloud Platform project.
* `<YOUR_BUCKET_ID>` - with the bucket id you created above.
* `<YOUR_MODEL_NAME>` - with your model name, such as "census"
* `<YOUR_VERSION>` - with your version name, such as "v1"
* `<ML_ENGINE_REGION>` - select a region from [here](https://cloud.google.com/ml-engine/docs/regions) or use the default 'us-central1'. The region is where the model will be deployed.
* `data.json` - a JSON file that contains the data used as input to your model’s predict method. (You'll create this in the code below)

## Download the data
The [Census Income Data Set](https://archive.ics.uci.edu/ml/datasets/Census+Income) that this sample
uses for training is hosted by the [UC Irvine Machine Learning
Repository](https://archive.ics.uci.edu/ml/datasets/):

 * Training file is `adult.data`
 * Evaluation file is `adult.test`


### Disclaimer
This dataset is provided by a third party. Google provides no representation,
warranty, or other guarantees about the validity or any other aspects of this dataset.

In [1]:
# Create a directory to hold the data
! mkdir census_data

# Download the data
! curl https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data --output census_data/adult.data
! curl https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test --output census_data/adult.test

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 3881k  100 3881k    0     0  9751k      0 --:--:-- --:--:-- --:--:-- 9751k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1956k  100 1956k    0     0  5526k      0 --:--:-- --:--:-- --:--:-- 5510k


# Part 1: Train/Save the model 
First, the data is loaded into a pandas DataFrame. Then a simple model is created with the training set. Lastly, the model is saved to a .bst file that can then be uploaded to AI Platform.

In [2]:
import xgboost as xgb
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# these are the column labels from the census data files
COLUMNS = (
    'age',
    'workclass',
    'fnlwgt',
    'education',
    'education-num',
    'marital-status',
    'occupation',
    'relationship',
    'race',
    'sex',
    'capital-gain',
    'capital-loss',
    'hours-per-week',
    'native-country',
    'income-level'
)

# categorical columns contain data that need to be turned into numerical values before being used by XGBoost
CATEGORICAL_COLUMNS = (
    'workclass',
    'education',
    'marital-status',
    'occupation',
    'relationship',
    'race',
    'sex',
    'native-country'
)

# load training set
with open('./census_data/adult.data', 'r') as train_data:
    raw_training_data = pd.read_csv(train_data, header=None, names=COLUMNS)
# remove column we are trying to predict ('income-level') from features list
train_features = raw_training_data.drop('income-level', axis=1)
# create training labels list
train_labels = (raw_training_data['income-level'] == ' >50K')


# load test set
with open('./census_data/adult.test', 'r') as test_data:
    raw_testing_data = pd.read_csv(test_data, names=COLUMNS, skiprows=1)
# remove column we are trying to predict ('income-level') from features list
test_features = raw_testing_data.drop('income-level', axis=1)
# create training labels list
test_labels = (raw_testing_data['income-level'] == ' >50K.')

# convert data in categorical columns to numerical values
encoders = {col:LabelEncoder() for col in CATEGORICAL_COLUMNS}
for col in CATEGORICAL_COLUMNS:
    train_features[col] = encoders[col].fit_transform(train_features[col])
for col in CATEGORICAL_COLUMNS:
    test_features[col] = encoders[col].fit_transform(test_features[col])

print('data loading complete')

data loading complete


In [3]:
# load data into DMatrix object
dtrain = xgb.DMatrix(train_features, train_labels)
dtest = xgb.DMatrix(test_features)

# train XGBoost model
bst = xgb.train({'objective':'reg:logistic'}, dtrain, 20)
bst.save_model('./model.bst')

print('model trained and saved')

model trained and saved


In [4]:
bst.predict(dtest)[[0, 2]]

array([0.00463351, 0.34860396], dtype=float32)

In [5]:
%env BUCKET_ID=bucket_test_ml_data

# Create BUCKET_NAME if it does not exist.
!gsutil mb gs://${BUCKET_ID}

env: BUCKET_ID=bucket_test_ml_data


In [7]:
!gsutil cp ./census_data/adult.data gs://${BUCKET_ID}/census_income/data/
!gsutil cp ./census_data/adult.test gs://${BUCKET_ID}/census_income/data/

Copying file://./census_data/adult.data [Content-Type=application/octet-stream]...
/ [1 files][  3.8 MiB/  3.8 MiB]                                                
Operation completed over 1 objects/3.8 MiB.                                      
Copying file://./census_data/adult.test [Content-Type=application/octet-stream]...
/ [1 files][  1.9 MiB/  1.9 MiB]                                                
Operation completed over 1 objects/1.9 MiB.                                      


In [8]:
!gcloud ai-platform local train \
  --package-path trainer \
  --module-name trainer.task \
  -- \
  --bucket_name $BUCKET_ID

Copying file://model.bst [Content-Type=application/octet-stream]...
/ [1 files][ 62.8 KiB/ 62.8 KiB]                                                
Operation completed over 1 objects/62.8 KiB.                                     


### Test Data Preparation
Before you begin predicting , you'll need to take some of the test data and prepare it, so that the test data can be used by the deployed model.

To get predictions, the data needs to be converted from a numpy array to a json array.

In [9]:
import json
import numpy as np

data = []
for i in range(len(test_features)):
  data.append([])
  for col in COLUMNS[:-1]: # ignore 'income-level' column as it isn't in feature set.
    # convert from numpy integers to standard integers
    data[i].append(int(np.uint64(test_features[col][i]).item()))

# write the test data to a json file
with open('data.json', 'w') as outfile:
  json.dump(data, outfile)

# get one person that makes <=50K and one that makes >50K to test our model.
print('Show a person that makes <=50K:')
print('\tFeatures: {0} --> Label: {1}\n'.format(data[0], test_labels[0]))

with open('less_than_50K.json', 'w') as outfile:
  json.dump(data[0], outfile)

  
print('Show a person that makes >50K:')
print('\tFeatures: {0} --> Label: {1}'.format(data[2], test_labels[2]))

with open('more_than_50K.json', 'w') as outfile:
  json.dump(data[2], outfile)

Show a person that makes <=50K:
	Features: [25, 4, 226802, 1, 7, 4, 7, 3, 2, 1, 0, 0, 40, 38] --> Label: False

Show a person that makes >50K:
	Features: [28, 2, 336951, 7, 12, 2, 11, 0, 4, 1, 0, 0, 40, 38] --> Label: True


In [10]:
!gcloud ai-platform local predict \
  --model-dir gs://${BUCKET_ID}/census_income/model/ \
  --json-instances less_than_50K.json \
  --framework xgboost \
  --signature-name 'census_income_model'
#   --verbosity debug

Copying gs://bucket_test_ml_data/census_income/model/test.txt...                
/ [2 files][ 62.8 KiB/ 62.8 KiB]                                                
Operation completed over 2 objects/62.8 KiB.                                     
Copying gs://bucket_test_ml_data/census_income/model/model.bst...
Copying gs://bucket_test_ml_data/census_income/model/test.txt...                
/ [2 files][ 62.8 KiB/ 62.8 KiB]                                                
Operation completed over 2 objects/62.8 KiB.                                     

[0.004633505828678608]


In [11]:
%env PROJECT_ID=mwpmltr
%env PACKAGE_PATH=trainer
%env MODULE_NAME=trainer.task
%env RUNTIME_VERSION=1.15
%env PYTHON_VERSION=3.7
%env REGION=us-central1

env: PROJECT_ID=mwpmltr
env: PACKAGE_PATH=trainer
env: MODULE_NAME=trainer.task
env: RUNTIME_VERSION=1.15
env: PYTHON_VERSION=3.7
env: REGION=us-central1


In [12]:
from datetime import datetime, timedelta
now=(datetime.now() + timedelta(hours=-5)).strftime("%Y%m%d_%H%M%S") # Central Time

In [13]:
%env JOB_NAME=census_income_job_{now}

!gcloud ai-platform jobs submit training $JOB_NAME  \
  --job-dir gs://${BUCKET_ID}/census_income/jobdir \
  --package-path $PACKAGE_PATH \
  --module-name $MODULE_NAME \
  --runtime-version $RUNTIME_VERSION \
  --python-version $PYTHON_VERSION \
  --region $REGION \
  -- \
  --bucket_name $BUCKET_ID
    
# Stream logs so that training is done before subsequent cells are run.
# Remove  '> /dev/null' to see step-by-step output of the model build steps.
!gcloud ai-platform jobs stream-logs $JOB_NAME > /dev/null

# Model should exit with status "SUCCEEDED"`
!gcloud ai-platform jobs describe $JOB_NAME --format="value(state)"

env: JOB_NAME=census_income_job_20200821_042248
Job [census_income_job_20200821_042248] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ai-platform jobs describe census_income_job_20200821_042248

or continue streaming the logs with the command

  $ gcloud ai-platform jobs stream-logs census_income_job_20200821_042248
jobId: census_income_job_20200821_042248
state: QUEUED
SUCCEEDED


# Part 2: Upload the model
To use your model with AI Platform, it needs to be uploaded to Google Cloud Storage (GCS). So next, we'll take your local ‘model.bst’ file and upload it to GCS via the Cloud SDK using gsutil.

Before continuing, make sure you're [properly authenticated](https://cloud.google.com/sdk/gcloud/reference/auth/) and have [access to the bucket](https://cloud.google.com/storage/docs/access-control/). This next command sets your project to the one specified above.

Note: If you get an error below, make sure the Cloud SDK is installed in the kernel's environment.

In [14]:
! gcloud config set project $PROJECT_ID

Updated property [core/project].


In [27]:
# task.py already has a code which copies the model to the bucket. However the below code can also be used to copy the model to GCS

#! gsutil cp model.bst gs://${BUCKET_ID}/census_income/model/

Note: The exact file name of of the exported model you upload to GCS is important! Your model must be named “model.joblib”, “model.pkl”, or “model.bst” with respect to the library you used to export it. This restriction ensures that the model will be safely reconstructed later by using the same technique for import as was used during export.

# Part 3: Create a model resource
AI Platform organizes your trained models using model and version resources. An AI Platform model is a container for the versions of your machine learning model. For more information on model resources and model versions look [here](https://cloud.google.com/ml-engine/docs/deploying-models#creating_a_model_version). 

At this step, you create a container that you can use to hold several different versions of your actual model.

In [28]:
%env MODEL_NAME= census_income_xgb_model_test
%env FRAMEWORK=xgboost
%env VERSION_NAME=v3

env: MODEL_NAME=census_income_xgb_model_test
env: FRAMEWORK=xgboost
env: VERSION_NAME=v3


In [29]:
! gcloud ai-platform models create $MODEL_NAME --regions $REGION

Using endpoint [https://ml.googleapis.com/]
ERROR: (gcloud.ai-platform.models.create) Resource in project [mwpmltr] is the subject of a conflict: Field: model.name Error: A model with the same name already exists.
- '@type': type.googleapis.com/google.rpc.BadRequest
  fieldViolations:
  - description: A model with the same name already exists.
    field: model.name


# Part 4: Create a model version

Now it’s time to get your model online and ready for predictions. The model version requires a few components as specified [here](https://cloud.google.com/ml-engine/reference/rest/v1/projects.models.versions#Version).

* __name__ - The name specified for the version when it was created. This will be the `VERSION_NAME` variable you declared at the beginning.
* __deployment Uri__ (curl) - The Google Cloud Storage location of the trained model used to create the version. This is the bucket that you uploaded the model to with your `BUCKET_ID`
* __runtime__ version - The Google Cloud ML runtime version to use for this deployment. This is set to 1.15
* __framework__ - The framework specifies if you are using: `TENSORFLOW`, `SCIKIT_LEARN`, `XGBOOST`. This is set to `XGBOOST`

Note: Runtime version 1.5 uses XGBoost 0.7. Runtime version 1.4 uses XGBoost 0.6. Please refer to the [runtime version dependency list](https://cloud.google.com/ml-engine/docs/runtime-version-list).

Note: It can take several minutes for you model to be available.

In [30]:
!gcloud ai-platform versions create $VERSION_NAME \
  --model $MODEL_NAME \
  --origin gs://${BUCKET_ID}/census_income/model/ \
  --runtime-version $RUNTIME_VERSION \
  --python-version $PYTHON_VERSION \
  --framework $FRAMEWORK

Using endpoint [https://ml.googleapis.com/]
Creating version (this might take a few minutes)......done.                    


In [19]:
! curl -X POST -H "Content-Type: application/json" \
   -d '{"name": "'$VERSION_NAME'", "deploymentUri": "gs://'$BUCKET_ID'/census_income/model/", "runtimeVersion": "1.15", "pythonVersion":"3.7", "framework": "XGBOOST"}' \
   -H "Authorization: Bearer `gcloud auth print-access-token`" \
    https://ml.googleapis.com/v1/projects/$PROJECT_ID/models/$MODEL_NAME/versions

{
  "error": {
    "code": 409,
    "message": "Field: version.name Error: A version with the same name already exists.",
    "status": "ALREADY_EXISTS",
    "details": [
      {
        "@type": "type.googleapis.com/google.rpc.BadRequest",
        "fieldViolations": [
          {
            "field": "version.name",
            "description": "A version with the same name already exists."
          }
        ]
      }
    ]
  }
}


# Part 5: Make an online prediction

It’s time to make  prediction with your newly deployed model. For making the online Predictions we will be using the json array that we created.


## Use Gcloud to make online predictions
Use the two people (as seen in the table) gathered in the previous step for the gcloud predictions.

| **Person** | age | workclass | fnlwgt | education | education-num | marital-status | occupation |
|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:
| **1** | 25| 4 | 226802 | 1 | 7 | 4 | 7 |
| **2** | 28| 2 | 336951 | 7 | 12 | 2 | 11 |

| **Person** | relationship | race | sex | capital-gain | capital-loss | hours-per-week | native-country || (Label) income-level|
|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:||:-:
| **1** | 3 | 2 | 1 | 0 | 0 | 40 | 38 || False (<=50K) |
| **2** | 0 | 4 | 1 | 0 | 0 | 40 | 38 || True (>50K) |


Creating a model version can take several minutes, check the status of your model version to see if it is available.

In [20]:
! gcloud ai-platform versions list --model $MODEL_NAME

Using endpoint [https://ml.googleapis.com/]
NAME  DEPLOYMENT_URI                                 STATE
v1    gs://bucket_test_ml_data/census_income/model/  READY
v2    gs://bucket_test_ml_data/census_income/model/  READY


Test the model with an online prediction using the data of a person who makes <=50K.

Note: If you see an error, the model from Part 4 may not be created yet as it takes several minutes for a new model version to be created.

In [21]:
! gcloud ai-platform predict --model $MODEL_NAME --version $VERSION_NAME --json-instances less_than_50K.json

Using endpoint [https://ml.googleapis.com/]
[0.004633505828678608]


Test the model with an online prediction using the data of a person who makes >50K.

In [22]:
! gcloud ai-platform predict --model $MODEL_NAME --version $VERSION_NAME --json-instances more_than_50K.json

Using endpoint [https://ml.googleapis.com/]
[0.3486039638519287]


Realise how the cells above return floats instead of booleans. Let's deal with that below so the output type of the predictions match those of the test set labels. We'll set the prediction to True if it's greater than 0.5 and to False otherwise.

## Use Python to make online predictions
We'll test the model with the entire test set and print out some of the results.

Note: If running notebook server on Compute Engine, make sure to ["allow full access to all Cloud APIs"](https://cloud.google.com/compute/docs/access/create-enable-service-accounts-for-instances#changeserviceaccountandscopes).

In [23]:
import googleapiclient.discovery
import os

PROJECT_ID = os.environ['PROJECT_ID']
VERSION_NAME = os.environ['VERSION_NAME']
MODEL_NAME = os.environ['MODEL_NAME']

service = googleapiclient.discovery.build('ml', 'v1')
name = 'projects/{}/models/{}'.format(PROJECT_ID, MODEL_NAME)
name += '/versions/{}'.format(VERSION_NAME)

response = service.projects().predict(
    name=name,
    body={'instances': data}
).execute()

if 'error' in response:
  print (response['error'])
else:
  online_results = response['predictions']
  # convert floats to booleans
  converted_responses = [x > 0.5 for x in online_results]
  # Print the first 10 responses
  for i, response in enumerate(converted_responses[:5]):
    print('Prediction: {}\tLabel: {}'.format(response, test_labels[i]))

Prediction: False	Label: False
Prediction: False	Label: False
Prediction: False	Label: True
Prediction: True	Label: True
Prediction: False	Label: False


# [Optional] Part 6: Verify Results
Let's visualise our predictions with a confusion matrix.

In [24]:
actual = pd.Series(test_labels, name='actual')
online = pd.Series(converted_responses, name='online')

pd.crosstab(actual,online)

online,False,True
actual,,
False,11790,645
True,1451,2395


Let's compare this with the confusion matrix of our local model.

In [25]:
local_results = bst.predict(dtest)
converted_local = [x > 0.5 for x in local_results]
local = pd.Series(converted_local, name='local')

pd.crosstab(actual,local)

local,False,True
actual,,
False,11790,645
True,1451,2395


Better, let's compare the raw results (pre-boolean-conversion) of our local and online models.

In [26]:
identical = 0
different = 0

for i in range(len(online_results)):
    if online_results[i] == local_results[i]:
        identical += 1
    else:
        different += 1
        
print('identical: {}, different: {}'.format(identical,different))

identical: 16281, different: 0


If all results are identical, it means we've successfully uploaded our local model to AI Platform and performed online predictions correctly.